In [713]:
import pandas as pd

import requests
import bs4

import re
import time

import warnings
warnings.filterwarnings('ignore')

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import pymysql

from tqdm import tqdm

from datetime import datetime

import threading
from queue import Queue
 
import os

In [328]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'
from plotly.subplots import make_subplots

In [442]:
str(datetime.today().year) +'/'+ str(datetime.today().month)

'2022/6'

# Function to get page of hisnyi & 永慶

In [443]:
def get_page_永慶():
    path   = r'/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/chromedriver'
    driver = webdriver.Chrome(path)
    website = r'https://buy.yungching.com.tw/region/%E4%BD%8F%E5%AE%85_p/%E5%8F%B0%E5%8C%97%E5%B8%82-_c/_rm/?pg=1'

    #這邊之後改成顯示等待
    driver.get(website)
    WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,'/html/body/main/div[2]/div[4]/ul')))
    分頁選單      = driver.find_element_by_xpath('//ul[@class="m-pagination-bd"]')
    總頁數        = 分頁選單.find_elements_by_xpath('./li')[-1]
    最後一頁_href = re.search(r'\d+\Z',總頁數.find_element_by_xpath('./a').get_attribute('href'))
    print(最後一頁_href[0])
    driver.close()
    #以上是幫助我確認有多少分頁
    return 最後一頁_href[0]


In [448]:
def get_page_信義():
    path   = r'/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/chromedriver'
    driver = webdriver.Chrome(path)
    website = r'https://www.sinyi.com.tw/buy/list/Taipei-city/Taipei-R-mrtline/03-mrt/default-desc/index'

    #這邊之後改成顯示等待
    driver.get(website)
    time.sleep(3)
    #WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="__next"]/div/div/span/div[4]/div/div/div[3]/div[2]/div[2]/div[21]')))
    分頁選單      = driver.find_element_by_xpath('//ul[@class="pagination"]')
    總頁數        = 分頁選單.find_elements_by_xpath('./li')[-2]
    最後一頁_href = 總頁數.text
    print(最後一頁_href)
    driver.close()
    return 最後一頁_href

In [371]:
#得到頁數
page_num      = get_page_永慶()
page_url_list = []
singe_page_website = r'https://buy.yungching.com.tw/region/%E4%BD%8F%E5%AE%85_p/%E5%8F%B0%E5%8C%97%E5%B8%82-_c/_rm/?pg='
for p in range(1,int(page_num)+1):
    page_url_list.append(singe_page_website+str(p))

146


In [444]:
page_num_hisnyi = get_page_信義()
page_url_list_hisnyi = []
singe_page_website_hisnyi = r'https://www.sinyi.com.tw/buy/list/Taipei-city/Taipei-R-mrtline/03-mrt/default-desc/'
for p in range(1,int(page_num_hisnyi)+1):
    page_url_list_hisnyi.append(singe_page_website_hisnyi+str(p))

<selenium.webdriver.remote.webelement.WebElement (session="b983b45ed02ee4768669dd5b099c8bfb", element="3129ca2d-046a-4b02-9112-81b2f7a23f77")>
382


# create a function to collect one page infos of 永慶 and hsinyi

In [529]:
def get_all_page_info_hsinyi(q,串列):
    Price_Month = str(datetime.today().year) +'_'+ str(datetime.today().month) + '_Price'
    def get_page_info_hsinyi(url = 'https://www.sinyi.com.tw/buy/list/Taipei-city/Taipei-R-mrtline/03-mrt/default-desc/1'):
        path   = r'/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/chromedriver'
        options = webdriver.ChromeOptions()
        prefs = {
        'profile.default_content_setting_values' :
            {
            'notifications' : 2
            }
        }
        options.add_experimental_option('prefs',prefs)
        driver = webdriver.Chrome(path,chrome_options=options)
        website = url

        #這邊之後改成顯示等待
        driver.get(website)
        time.sleep(3)
        #WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframeCssSelector")))
        #WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="__next"]/div/div/span/div[4]/div/div/div[3]/div[2]/div[2]/div[21]')))
        房屋列表      = driver.find_element_by_xpath('//div[@class="buy-list-frame"]')
        單一房屋_all  = 房屋列表.find_elements_by_xpath('./div[@class="buy-list-item "]')
        one_page_house_info = []
        for 單一房屋 in 單一房屋_all:
            house_href    = 單一房屋.find_element_by_xpath('./a').get_attribute('href')
            house_detail  = 單一房屋.find_element_by_xpath('./a/div/div[@class="LongInfoCard_Type"]/div[@class="LongInfoCard_TypeWeb"]')
            house_left    = house_detail.find_element_by_xpath('./div[@class="LongInfoCard_Type_Left"]')
            #print(house_left)
            house_title   = house_left.find_element_by_xpath('./div[@class="LongInfoCard_Type_Name"]').text
            house_address = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_Address"]/span')[0].text
            house_age     = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_Address"]/span')[1].text
            house_type    = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_Address"]/span')[2].text
            house_建坪    = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_HouseInfo"]/span')[0].text
            house_主陽    = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_HouseInfo"]/span')[1].text
            house_格局    = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_HouseInfo"]/span')[2].text
            house_floor  = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_HouseInfo"]/span')[3].text
            house_車位    = house_left.find_element_by_xpath('./span[@class="LongInfoCard_Type_Parking"]').text

            #只跟價格有關
            house_right   = house_detail.find_element_by_xpath('./div[@class="LongInfoCard_Type_Right"]')
            house_price   = house_right.find_elements_by_xpath('./div')[1]
            house_p       = house_price.find_elements_by_xpath('./span')
            if len(house_p) == 2:
                house_p_final = house_p[0].text + house_p[1].text
            elif len(house_p) == 3:
                house_p_final = house_p[0].text + house_p[1].text + house_p[2].text
            one_page_house_info.append([house_address,house_href,house_title,house_p_final,house_type,house_age,house_floor,house_建坪,house_主陽,house_格局,house_車位])
        q.put(one_page_house_info)
        driver.close()

       #if len(pd.DataFrame(one_page_house_info)) != 0:
       #    return (pd.DataFrame(one_page_house_info))
       #else:
       #    print(url)
       #    return pd.DataFrame({})

    #all_info_df = pd.DataFrame({})
    for p in tqdm(串列):
        Single_Page = get_page_info_hsinyi(p)
        #all_info_df = all_info_df.append(Single_Page)
    return 'Done'

In [477]:
def get_page_info_hsinyi_test(q,url = 'https://www.sinyi.com.tw/buy/list/Taipei-city/Taipei-R-mrtline/03-mrt/default-desc/5'):
        path   = r'/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/chromedriver'
        options = webdriver.ChromeOptions()
        prefs = {
        'profile.default_content_setting_values' :
            {
            'notifications' : 2
            }
        }
        options.add_experimental_option('prefs',prefs)
        driver = webdriver.Chrome(path,chrome_options=options)
        website = url

        #這邊之後改成顯示等待
        driver.get(website)
        time.sleep(3)

        房屋列表      = driver.find_element_by_xpath('//div[@class="buy-list-frame"]')
        單一房屋_all  = 房屋列表.find_elements_by_xpath('./div[@class="buy-list-item "]')
        one_page_house_info = []
        for 單一房屋 in 單一房屋_all:
            house_href    = 單一房屋.find_element_by_xpath('./a').get_attribute('href')
            house_detail  = 單一房屋.find_element_by_xpath('./a/div/div[@class="LongInfoCard_Type"]/div[@class="LongInfoCard_TypeWeb"]')
            house_left    = house_detail.find_element_by_xpath('./div[@class="LongInfoCard_Type_Left"]')
            #print(house_left)
            house_title   = house_left.find_element_by_xpath('./div[@class="LongInfoCard_Type_Name"]').text
            house_address = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_Address"]/span')[0].text
            house_age     = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_Address"]/span')[1].text
            house_type    = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_Address"]/span')[2].text
            house_建坪    = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_HouseInfo"]/span')[0].text
            house_主陽    = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_HouseInfo"]/span')[1].text
            house_格局    = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_HouseInfo"]/span')[2].text
            house_floor  = house_left.find_elements_by_xpath('./div[@class="LongInfoCard_Type_HouseInfo"]/span')[3].text
            house_車位    = house_left.find_element_by_xpath('./span[@class="LongInfoCard_Type_Parking"]').text

            #只跟價格有關
            house_right   = house_detail.find_element_by_xpath('./div[@class="LongInfoCard_Type_Right"]')
            house_price   = house_right.find_elements_by_xpath('./div')[1]
            house_p       = house_price.find_elements_by_xpath('./span')
            if len(house_p) == 2:
                house_p_final = house_p[0].text + house_p[1].text
            elif len(house_p) == 3:
                house_p_final = house_p[0].text + house_p[1].text + house_p[2].text
            one_page_house_info.append([house_address,house_href,house_title,house_p_final,house_type,house_age,house_floor,house_建坪,house_主陽,house_格局,house_車位])
        q.put(one_page_house_info)
        driver.close()
        if len(pd.DataFrame(one_page_house_info)) != 0:
            return (pd.DataFrame(one_page_house_info))
        else:
            return url

In [710]:
#queue = Queue()
queue2 = Queue()
#j1 = threading.Thread(target = get_all_page_info_hsinyi, args=(queue,page_url_list_hisnyi[361:371],))
j2 = threading.Thread(target = get_all_page_info_hsinyi, args=(queue2,page_url_list_hisnyi[381:],))
#j1.start()
j2.start()
#j1.join()
j2.join()

100%|██████████| 1/1 [00:32<00:00, 32.02s/it]


In [704]:
a = pd.DataFrame()
while not queue.empty():
    c = queue.get()
    a = a.append((pd.DataFrame(c)))
    print('-')
    #a = a.append(pd.DataFrame(queue.get()))

-
-
-
-
-
-
-
-
-
-


In [711]:
b = pd.DataFrame()
while not queue2.empty():
    c = queue2.get()
    b = b.append((pd.DataFrame(c)))
    print('-')
    #a = a.append(pd.DataFrame(queue.get()))

-


In [712]:
#a.columns = ['Address','href','Title','2022_6_Price','Type','Age','Floor','地坪','主+(陽)','格局','車位']
#a.to_csv(r'/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/output/信義361-371頁.csv',index=False,header=True,encoding='utf-8-sig')
b.columns = ['Address','href','Title','2022_6_Price','Type','Age','Floor','地坪','主+(陽)','格局','車位']
b.to_csv(r'/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/output/信義381-頁.csv',index=False,header=True,encoding='utf-8-sig')

In [192]:
def get_all_page_info_永慶(串列):
    Price_Month = str(datetime.today().year) +'_'+ str(datetime.today().month) + '_Price'
    def get_page_info_永慶(url):
        測試網頁     = requests.get(url)
        bs4_web     = bs4.BeautifulSoup(測試網頁.text,'lxml')
        房屋列表     = bs4_web.find('div',class_='l-main-list').find('ul',class_='l-item-list')
        單一房屋_all = 房屋列表.find_all('li',class_='m-list-item')
        one_page_house_info = []
        for 單一房屋 in 單一房屋_all:

            house_href      = 'https://buy.yungching.com.tw/'+單一房屋.find('a').get('href')
            house_title     = 單一房屋.find('a').get('title').split(' ')[0]
            house_address   = 單一房屋.find('a').get('title').split(' ')[1]
            house_price     = 單一房屋.find('div',class_='item-price').find('div',class_='price').text

            house_info_list = [house_href,house_title,house_address,house_price]

            house_details   = 單一房屋.find('ul').find_all('li')
            for detail in house_details:
                #print(detail.text)
                house_info_list.append(detail.text)

            one_page_house_info.append(house_info_list)

            
        單一頁面資訊_df = pd.DataFrame(one_page_house_info,columns=['href','Title','Address',Price_Month,'Type','Age','Floor','地坪','主+(陽)','建物','格局','加蓋','車位'])
        for col in ['Age','Floor','格局','加蓋','車位']:
            單一頁面資訊_df[col] = 單一頁面資訊_df[col].apply(lambda x: x.replace('\r\n',''))
        
        return 單一頁面資訊_df

    all_info_df = pd.DataFrame()
    for p in tqdm(串列):
        Single_Page = get_page_info_永慶(p)
        all_info_df = all_info_df.append(Single_Page)

    return all_info_df

In [197]:
All_Data_永慶 = get_all_page_info_永慶(page_url_list)

100%|██████████| 148/148 [02:37<00:00,  1.07s/it]


# DataClean

In [883]:
def data_clean_信義(dataframe):

    def get_Age(x):
        if x == '--':
            return '未提供'
        elif x == '預售':
            return '預售'
        else:
            return x.replace('年','')
        
    def get_房(x):
        if re.search(r'(\d)(房)',x) != None:
            return int(re.search(r'(\d)(房)',x).group(1))
        else:
            return 0
        
    def get_廳(x):
        if re.search(r'(\d)(廳)',x) != None:
            return int(re.search(r'(\d)(廳)',x).group(1))
        else:
            return 0

    def get_衛(x):
        if re.search(r'(\d+)廳(\d+.{0,1}\d{0,2})衛',x) != None:
            return float(re.search(r'(\d+)廳(\d+.{0,1}\d{0,2})衛',x).group(2))
        else:
            return 0

    def get_室(x):
        if re.search(r'(\d+)室',x) != None:
            return int(re.search(r'(\d+)(室)',x).group(1))
        else:
            return 0

    def get_price(x):
        y = x.replace(',','')
        if re.match(r'\d+萬\d+萬',y) != None:
            原價 = re.match(r'(\d+)(萬)(\d+)(萬)',y).group(1)
            降價 = re.match(r'(\d+)(萬)(\d+)(萬)',y).group(3)
            return 降價, 原價
        elif re.match(r'\d+萬',y) != None:
            原價 = re.match(r'(\d+)(萬)',y).group(1)
            return 原價, '無'
        else:
            return '無', '無'

    def 車位轉換(x):
        if x == '':
            return 'N'
        else:
            return 'Y'


    Price_Month = str(datetime.today().year) +'_'+ str(datetime.today().month) + '_Price'
    Price_Month_exist = str(datetime.today().year) +'_'+ str(datetime.today().month) + '_Price_較高價格(if exiset)'
    dataframe                  = dataframe[(dataframe['Address'].str.contains('台北市'))|(dataframe['Address'].str.contains('台北縣'))]
    dataframe['車位']          = dataframe['車位'].fillna('')
    dataframe['City']          = dataframe['Address'].apply(lambda x:re.match(r'\D{2,3}[市|縣]',x)[0])
    dataframe['District']      = dataframe['Address'].apply(lambda x:re.match(r'(\D{2,3}[市|縣])(\D{2,3}區)',x).group(2))
    dataframe['Road']          = dataframe['Address'].apply(lambda x:re.match(r'(\D{2,3}[市|縣])(\D{2,3}區)(\D+)',x).group(3) if re.match(r'(\D+[市|縣])(\D+區)(\D+)',x) != None else 'X')
    dataframe[Price_Month_exist]     = dataframe[Price_Month].apply(lambda x: get_price(x)[1])
    dataframe[Price_Month]     = dataframe[Price_Month].apply(lambda x: get_price(x)[0])
    dataframe['Age']           = dataframe['Age'].apply(lambda x:get_Age(x))
    dataframe['地坪']           = dataframe['地坪'].apply(lambda x:re.search(r'\d+.\d+',x)[0]  if re.search(r'\d+.\d+',x) != None else '未提供')
    dataframe['主+(陽)_f']      = dataframe['主+(陽)'].apply(lambda x:re.search(r'\d+.\d+',x)[0] if re.search(r'\d+.\d+',x) != None else '未提供')
    dataframe['加蓋']          = dataframe['格局'].apply(lambda x: 'Y' if ('含加蓋') in x else 'N')
    dataframe['房']            = dataframe['格局'].apply(lambda x: get_房(x) + get_室(x))
    dataframe['廳']            = dataframe['格局'].apply(lambda x: get_廳(x))
    dataframe['衛']            = dataframe['格局'].apply(lambda x: get_衛(x))
    dataframe['車位_f']        = dataframe['車位'].apply(lambda x: 車位轉換(x))
    return dataframe

In [884]:
All_Data_Clean_信義 = pd.DataFrame({})
output_folder = os.path.join(os.getcwd(),'output')
for file in os.listdir(output_folder):
    if '信義' in file and '全資料' not in file: 
        c = pd.read_csv(output_folder + '/' + file)
        clean_c = data_clean_信義(c)
        All_Data_Clean_信義 = All_Data_Clean_信義.append(clean_c)

In [922]:
output_folder = os.path.join(os.getcwd(),'output')
for file in os.listdir(output_folder):
    if '信義' in file and '全資料' not in file: 
        c = pd.read_csv(output_folder + '/' + file)
        if '台北市內湖區麗山街' in c['Address'].values:
            print(output_folder + '/' + file)

/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/output/信義321-331頁.csv
/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/output/信義351-361頁.csv
/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/output/信義341-351頁.csv
/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/output/信義21-31頁.csv
/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/output/信義311-321頁.csv


In [885]:
All_Data_Clean_信義.reset_index(drop=True,inplace=True)
All_Data_Clean_信義.to_csv(r'/Users/andyhsu/Documents/GitHub/Andy-Portfolio/信義房屋/output/信義全資料/信義全資料.csv',index=False,header=True,encoding='utf-8-sig')

In [298]:
def data_clean_永慶(dataframe):
    Price_Month = str(datetime.today().year) +'_'+ str(datetime.today().month) + '_Price'
    dataframe                  = dataframe[(dataframe['Address'].str.contains('台北市'))|(dataframe['Address'].str.contains('台北縣'))]
    dataframe['City']          = dataframe['Address'].apply(lambda x:re.match(r'\D{2,3}[市|縣]',x)[0])
    dataframe['District']      = dataframe['Address'].apply(lambda x:re.match(r'(\D{2,3}[市|縣])(\D{2,3}區)',x).group(2))
    dataframe['Road']          = dataframe['Address'].apply(lambda x:re.match(r'(\D{2,3}[市|縣])(\D{2,3}區)(\D+)',x).group(3) if re.match(r'(\D+[市|縣])(\D+區)(\D+)',x) != None else 'X')
    dataframe[Price_Month]     = dataframe[Price_Month].apply(lambda x: int(x.replace('萬','').replace(',','')))
    dataframe['Age']           = dataframe['Age'].apply(lambda x: float(x.replace('年','')) if x != '\n' else 1.0)
    dataframe['地坪']           = dataframe['地坪'].apply(lambda x:re.search(r'\d+.\d+',x)[0]  if re.search(r'\d+.\d+',x) != None else '未提供')
    dataframe['主+(陽)_f']      = dataframe['主+(陽)'].apply(lambda x:re.search(r'\d+.\d+',x)[0] if re.search(r'\d+.\d+',x) != None else '未提供')
    dataframe['建物']           = dataframe['建物'].apply(lambda x:re.search(r'\d+.\d+',x)[0] if re.search(r'\d+.\d+',x) != None else '未提供')
    dataframe['房']            = dataframe['格局'].apply(lambda x:re.match(r'(\d+)房\(室\)(\d+)廳(\d+.{0,1}\d{0,2})衛',x).group(1) if re.match(r'(\d+)房\(室\)(\d+)廳(\d+.{0,1}\d{0,2})衛',x) != None else '未提供')
    dataframe['廳']            = dataframe['格局'].apply(lambda x:re.match(r'(\d+)房\(室\)(\d+)廳(\d+.{0,1}\d{0,2})衛',x).group(2) if re.match(r'(\d+)房\(室\)(\d+)廳(\d+.{0,1}\d{0,2})衛',x) != None else '未提供')
    dataframe['衛']            = dataframe['格局'].apply(lambda x:re.match(r'(\d+)房\(室\)(\d+)廳(\d+.{0,1}\d{0,2})衛',x).group(3) if re.match(r'(\d+)房\(室\)(\d+)廳(\d+.{0,1}\d{0,2})衛',x) != None else '未提供')
    def 車位轉換(x):
        if x == '\n':
            return '沒附車位'
        elif x == '(含車位)':
            return 0
        else:
            if re.search(r'\d+.\d+',x) != None:
                return re.search(r'\d+.\d+',x)[0]
            else:
                return '未提供'
    dataframe['車位_f']        = dataframe['車位'].apply(lambda x:車位轉換(x))
    dataframe.reset_index(drop=True,inplace=True)
    #讓Address當第一個
    dataframe = dataframe.set_index('Address').reset_index(drop=False)
    return dataframe

In [299]:
All_Data_Clean_永慶 = data_clean_永慶(All_Data_永慶)

In [278]:
All_Data_Clean_永慶['District'].unique()

array(['中正區', '大安區', '士林區', '中山區', '信義區', '內湖區', '南港區', '松山區', '萬華區',
       '北投區', '大同區', '文山區'], dtype=object)

In [300]:
Month = str(datetime.today().year) +'_'+ str(datetime.today().month)
file  = r'output/'  + Month + '.csv'
print(file) 
All_Data_Clean_永慶.to_csv(file,index=False,header=True,encoding='utf-8-sig')

output/2022_6.csv


# Create DB

In [245]:
db = pymysql.connect(host = "localhost", user = "root", password = "airflow")
cursor = db.cursor()
cursor.execute(" CREATE DATABASE `HsinYiAssetDB_CHI` CHARACTER SET utf8 COLLATE utf8_general_ci;")
db.close()

In [889]:
All_Data_Clean_信義.columns

Index(['Address', 'href', 'Title', '2022_6_Price', 'Type', 'Age', 'Floor',
       '地坪', '主+(陽)', '格局', '車位', 'City', 'District', 'Road',
       '2022_6_Price_較高價格(if exiset)', '主+(陽)_f', '加蓋', '房', '廳', '衛', '車位_f'],
      dtype='object')

# hsinyi table

In [912]:
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "HsinYiAssetDB_CHI" )
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS HSINYI_INFO")
sql = """CREATE TABLE HSINYI_INFO (
         Address  CHAR(150) NOT NULL,
         href CHAR(150),
         Title CHAR(40),
         Price INT(7),
         Type CHAR(10),
         Age CHAR(9),
         Floor CHAR(20),
         地坪 CHAR(15),
         主陽台 CHAR(20),
         格局 CHAR(25),
         加蓋 CHAR(1),
         車位 CHAR(20),
         City CHAR(3),
         District CHAR(5),
         Road CHAR(10),
         Price_High CHAR(9),
         主陽台_f CHAR(7),
         房 CHAR(5),
         廳 CHAR(5),
         衛 CHAR(5),
         車位_f CHAR(2)
         )
         """
cursor.execute(sql)
print("CREATE TABLE OK")
cursor.close()
db.close()

CREATE TABLE OK


In [913]:
records_hsinyi = []
for x in All_Data_Clean_信義.values:
    records_hsinyi.append(tuple(x))

In [914]:
All_Data_Clean_信義.columns

Index(['Address', 'href', 'Title', '2022_6_Price', 'Type', 'Age', 'Floor',
       '地坪', '主+(陽)', '格局', '車位', 'City', 'District', 'Road',
       '2022_6_Price_較高價格(if exiset)', '主+(陽)_f', '加蓋', '房', '廳', '衛', '車位_f'],
      dtype='object')

In [915]:
records_hsinyi[0]

('台北市大安區泰順街',
 'https://www.sinyi.com.tw/buy/house/75430Q/?breadcrumb=list',
 '台電大樓站泰順街花園庭院',
 '6280',
 '公寓',
 '39.6',
 'B1-1樓/5樓',
 '50.29',
 '主建物29.38',
 '3房2廳2衛1室',
 '',
 '台北市',
 '大安區',
 '泰順街',
 '無',
 '29.38',
 'N',
 4,
 2,
 2.0,
 'N')

In [916]:
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "HsinYiAssetDB_CHI" )

#要新增或修改資料庫內容要呼叫的指標物件
mycursor = db.cursor()


sql = """INSERT INTO HSINYI_INFO (Address, href, Title, Price, Type, Age, Floor, 地坪, 主陽台, 格局, 車位, City, District, Road, Price_High, 主陽台_f, 加蓋, 房, 廳, 衛, 車位_f) VALUES \
    (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
val = records_hsinyi


mycursor.executemany(sql, val)

#提交任務
db.commit()
print(mycursor.rowcount, "was inserted.")

mycursor.close()
db.close()

6199 was inserted.


# 永慶table

In [301]:
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "HsinYiAssetDB_CHI" )
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS NEWEST_INFO")
sql = """CREATE TABLE NEWEST_INFO (
         Address  CHAR(50) NOT NULL,
         href CHAR(150),
         Title CHAR(40),
         Price INT(7),
         Type CHAR(10),
         Age FLOAT(4,1),
         Floor CHAR(20),
         地坪 CHAR(15),
         主陽台 CHAR(20),
         建物 CHAR(15),
         格局 CHAR(15),
         加蓋 CHAR(20),
         車位 CHAR(20),
         City CHAR(3),
         District CHAR(5),
         Road CHAR(10),
         主陽台_f CHAR(7),
         房 CHAR(5),
         廳 CHAR(5),
         衛 CHAR(5),
         車位_f CHAR(10)
         )
         """
cursor.execute(sql)
print("CREATE TABLE OK")
cursor.close()
db.close()

CREATE TABLE OK


In [302]:
records = []
for x in All_Data_Clean_永慶.values:
    records.append(tuple(x))

In [303]:
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "HsinYiAssetDB_CHI" )

#要新增或修改資料庫內容要呼叫的指標物件
mycursor = db.cursor()


sql = """INSERT INTO NEWEST_INFO (Address, href, Title, Price, Type, Age, Floor, 地坪, 主陽台, 建物, 格局, 加蓋, 車位, City, District, Road, 主陽台_f, 房, 廳, 衛, 車位_f) VALUES \
    (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
val = records


mycursor.executemany(sql, val)

#提交任務
db.commit()
print(mycursor.rowcount, "was inserted.")

mycursor.close()
db.close()

4404 was inserted.


In [305]:
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "HsinYiAssetDB_CHI" )
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS PRICE_TREND")
sql = """CREATE TABLE PRICE_TREND (
         Address  CHAR(50) NOT NULL,
         href CHAR(150),
         Title CHAR(40),
         Price_2022_06 INT(7)
         )
         """
cursor.execute(sql)
print("CREATE TABLE OK")
cursor.close()
db.close()

CREATE TABLE OK


In [307]:
price_records = []
for x in All_Data_Clean_永慶.values:
    price_records.append(tuple(x[:4]))

In [311]:
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "HsinYiAssetDB_CHI" )

#要新增或修改資料庫內容要呼叫的指標物件
mycursor = db.cursor()


sql = """INSERT INTO PRICE_TREND (Address, href, Title, Price_2022_06) VALUES \
    (%s, %s, %s, %s)"""
val = price_records


mycursor.executemany(sql, val)

#提交任務
db.commit()
print(mycursor.rowcount, "was inserted.")

mycursor.close()
db.close()

4404 was inserted.
